## 0. Import Modules and Data

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime as dt

In [2]:
import Portfolio as port # This is from the Portfolio.py file with the different classes defined

## 1. Set-up Portfolio Strategy

In [3]:
# Define start date and end date

start_date = '2015-01-01'

end_date = '2020-01-05'

In [4]:
# Define Shares

# Provide the name of the ticker and type (Equity or Bond)

voo = port.Share('VOO', 'Equity', start_date,end_date)
bnd = port.Share('BND', 'Bond', start_date, end_date)

In [6]:
voo.get_value('2019-01-01')

229.99

In [7]:
voo.get_value('2019-10-31')

278.55

In [8]:
# Need a way to group the different shares together

shares_list = [voo, bnd]

shares_dict = {}

for share in shares_list:
    shares_dict[share] = share.type


In [9]:
shares_dict

{<Portfolio.Share at 0x7ff3cf28f0b8>: 'Equity',
 <Portfolio.Share at 0x7ff3cf28f080>: 'Bond'}

In [10]:
# Define Strategy

# Provide the equity distribution, the bond distriubtion, cash distribution, and the threshold
strat = port.Strategy(50,50,0,5)

In [25]:
# Run portfolio with set start and end dates

portfolio = port.Portfolio(shares_dict)

portfolio.initial_buy(500, strat, start_date)

In [26]:
portfolio.get_asset_values(start_date)

In [27]:
portfolio.asset_split

{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}

In [28]:
# Run the portfolio over a series of months

time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

for day in time_period:
    print(day)
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
    print(portfolio.asset_split)
    if portfolio.asset_split['Equities'] > strat.equity_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*((portfolio.asset_split['Equities']-strat.equity_distribution)/100)
        sell_amt_per = sell_amt/len(portfolio.equities)
        for share in portfolio.equities: # sell equities and buy more bonds
            portfolio.sell(share, sell_amt_per, day)
        for share in portfolio.bonds:
            portfolio.buy(share, sell_amt_per, day)
               
    if portfolio.asset_split['Bonds'] > strat.bond_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*(portfolio.asset_split['Bonds']-strat.bond_distribution)
        sell_amt_per = sell_amt/len(portfolio.bonds)
        for share in portfolio.bonds: # sell bonds and buy more equities
            portfolio.sell(share, sell_amt_per, day)
        for share in portfolio.bonds:
            portfolio.buy(share, sell_amt_per, day)
        
    

2015-01-01 00:00:00
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-02 00:00:00
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-03 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-04 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-05 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-06 00:00:00
{'Equities': 49.16513484755205, 'Bonds': 50.83486515244794, 'Cash': 0.0}
2015-01-07 00:00:00
{'Equities': 49.460491131767945, 'Bonds': 50.53950886823204, 'Cash': 0.0}
2015-01-08 00:00:00
{'Equities': 49.934970195913024, 'Bonds': 50.06502980408698, 'Cash': 0.0}
2015-01-09 00:00:00
{'Equities': 49.68145734582606, 'Bonds': 50.31854265417394, 'Cash': 0.0}
2015-01-10 00:00:00
{'Equities': 49.462145782702805, 'Bonds': 50.537854217297195, 'Cash': 0.0}
2015-01-11 00:00:00
{'Equities': 49.462145782702805, 'Bonds': 50.537854217297195, 'Cash': 0.0}
2015-01-12 00:00:00
{'E

2015-05-11 00:00:00
{'Equities': 50.83752835227373, 'Bonds': 49.162471647726264, 'Cash': 0.0}
2015-05-12 00:00:00
{'Equities': 50.7954166801886, 'Bonds': 49.20458331981141, 'Cash': 0.0}
2015-05-13 00:00:00
{'Equities': 50.815420079511306, 'Bonds': 49.184579920488694, 'Cash': 0.0}
2015-05-14 00:00:00
{'Equities': 51.026178832867195, 'Bonds': 48.9738211671328, 'Cash': 0.0}
2015-05-15 00:00:00
{'Equities': 50.951489991765044, 'Bonds': 49.048510008234956, 'Cash': 0.0}
2015-05-16 00:00:00
{'Equities': 51.11959218410714, 'Bonds': 48.88040781589285, 'Cash': 0.0}
2015-05-17 00:00:00
{'Equities': 51.11959218410714, 'Bonds': 48.88040781589285, 'Cash': 0.0}
2015-05-18 00:00:00
{'Equities': 51.11959218410714, 'Bonds': 48.88040781589285, 'Cash': 0.0}
2015-05-19 00:00:00
{'Equities': 51.16168638779015, 'Bonds': 48.83831361220985, 'Cash': 0.0}
2015-05-20 00:00:00
{'Equities': 51.1450426721374, 'Bonds': 48.8549573278626, 'Cash': 0.0}
2015-05-21 00:00:00
{'Equities': 51.1412451317816, 'Bonds': 48.85875

2015-09-10 00:00:00
{'Equities': 49.266068116571944, 'Bonds': 50.73393188342806, 'Cash': 0.0}
2015-09-11 00:00:00
{'Equities': 49.31802256947305, 'Bonds': 50.68197743052695, 'Cash': 0.0}
2015-09-12 00:00:00
{'Equities': 49.21745114532532, 'Bonds': 50.78254885467468, 'Cash': 0.0}
2015-09-13 00:00:00
{'Equities': 49.21745114532532, 'Bonds': 50.78254885467468, 'Cash': 0.0}
2015-09-14 00:00:00
{'Equities': 49.21745114532532, 'Bonds': 50.78254885467468, 'Cash': 0.0}
2015-09-15 00:00:00
{'Equities': 49.64549294617653, 'Bonds': 50.35450705382347, 'Cash': 0.0}
2015-09-16 00:00:00
{'Equities': 49.87838124640987, 'Bonds': 50.121618753590134, 'Cash': 0.0}
2015-09-17 00:00:00
{'Equities': 49.67149556529521, 'Bonds': 50.32850443470479, 'Cash': 0.0}
2015-09-18 00:00:00
{'Equities': 49.19366691084458, 'Bonds': 50.80633308915543, 'Cash': 0.0}
2015-09-19 00:00:00
{'Equities': 49.281371827687, 'Bonds': 50.718628172313004, 'Cash': 0.0}
2015-09-20 00:00:00
{'Equities': 49.281371827687, 'Bonds': 50.7186281

2016-02-20 00:00:00
{'Equities': 49.01092664895592, 'Bonds': 50.98907335104407, 'Cash': 0.0}
2016-02-21 00:00:00
{'Equities': 49.01092664895592, 'Bonds': 50.98907335104407, 'Cash': 0.0}
2016-02-22 00:00:00
{'Equities': 49.01092664895592, 'Bonds': 50.98907335104407, 'Cash': 0.0}
2016-02-23 00:00:00
{'Equities': 48.68032372154051, 'Bonds': 51.31967627845948, 'Cash': 0.0}
2016-02-24 00:00:00
{'Equities': 48.77938370197183, 'Bonds': 51.22061629802816, 'Cash': 0.0}
2016-02-25 00:00:00
{'Equities': 49.041520749927706, 'Bonds': 50.958479250072294, 'Cash': 0.0}
2016-02-26 00:00:00
{'Equities': 49.019795892939506, 'Bonds': 50.9802041070605, 'Cash': 0.0}
2016-02-27 00:00:00
{'Equities': 48.78021657913304, 'Bonds': 51.21978342086696, 'Cash': 0.0}
2016-02-28 00:00:00
{'Equities': 48.78021657913304, 'Bonds': 51.21978342086696, 'Cash': 0.0}
2016-02-29 00:00:00
{'Equities': 48.78021657913304, 'Bonds': 51.21978342086696, 'Cash': 0.0}
2016-03-01 00:00:00
{'Equities': 49.50601912070434, 'Bonds': 50.4939

2016-07-03 00:00:00
{'Equities': 50.04881214094805, 'Bonds': 49.95118785905195, 'Cash': 0.0}
2016-07-04 00:00:00
{'Equities': 50.04881214094805, 'Bonds': 49.95118785905195, 'Cash': 0.0}
2016-07-05 00:00:00
{'Equities': 50.04881214094805, 'Bonds': 49.95118785905195, 'Cash': 0.0}
2016-07-06 00:00:00
{'Equities': 50.151833808170245, 'Bonds': 49.848166191829755, 'Cash': 0.0}
2016-07-07 00:00:00
{'Equities': 50.17623700454791, 'Bonds': 49.82376299545209, 'Cash': 0.0}
2016-07-08 00:00:00
{'Equities': 50.48820752253869, 'Bonds': 49.51179247746133, 'Cash': 0.0}
2016-07-09 00:00:00
{'Equities': 50.64887208609338, 'Bonds': 49.351127913906616, 'Cash': 0.0}
2016-07-10 00:00:00
{'Equities': 50.64887208609338, 'Bonds': 49.351127913906616, 'Cash': 0.0}
2016-07-11 00:00:00
{'Equities': 50.64887208609338, 'Bonds': 49.351127913906616, 'Cash': 0.0}
2016-07-12 00:00:00
{'Equities': 50.924968063473244, 'Bonds': 49.07503193652675, 'Cash': 0.0}
2016-07-13 00:00:00
{'Equities': 50.87166635871239, 'Bonds': 49.

...

{'Equities': 51.787742913305166, 'Bonds': 43.629334849072634, 'Cash': 4.582922237622196}
2018-10-16 00:00:00
{'Equities': 52.29620151153115, 'Bonds': 43.17236287486986, 'Cash': 4.53143561359899}
2018-10-17 00:00:00
{'Equities': 52.34543475274488, 'Bonds': 43.120447849648556, 'Cash': 4.534117397606571}
2018-10-18 00:00:00
{'Equities': 52.00688174077806, 'Bonds': 43.42413651677262, 'Cash': 4.5689817424493215}
2018-10-19 00:00:00
{'Equities': 51.99882633498599, 'Bonds': 43.429300425721266, 'Cash': 4.571873239292757}
2018-10-20 00:00:00
{'Equities': 51.904067916542616, 'Bonds': 43.51343566858571, 'Cash': 4.582496414871675}
2018-10-21 00:00:00
{'Equities': 51.904067916542616, 'Bonds': 43.51343566858571, 'Cash': 4.582496414871675}
2018-10-22 00:00:00
{'Equities': 51.904067916542616, 'Bonds': 43.51343566858571, 'Cash': 4.582496414871675}
2018-10-23 00:00:00
{'Equities': 51.7215834441488, 'Bonds': 43.68440909841681, 'Cash': 4.594007457434387}
2018-10-24 00:00:00
{'Equities': 50.90373269173215,

{'Equities': 51.691568936944265, 'Bonds': 43.83516244928127, 'Cash': 4.473268613774461}
2019-02-27 00:00:00
{'Equities': 51.72424337765932, 'Bonds': 43.79739335395973, 'Cash': 4.47836326838097}
2019-02-28 00:00:00
{'Equities': 51.69133035299344, 'Bonds': 43.82266942052366, 'Cash': 4.486000226482917}
2019-03-01 00:00:00
{'Equities': 51.96959515014555, 'Bonds': 43.551042780411, 'Cash': 4.479362069443455}
2019-03-02 00:00:00
{'Equities': 51.83367116515485, 'Bonds': 43.681461600922816, 'Cash': 4.484867233922344}
2019-03-03 00:00:00
{'Equities': 51.83367116515485, 'Bonds': 43.681461600922816, 'Cash': 4.484867233922344}
2019-03-04 00:00:00
{'Equities': 51.83367116515485, 'Bonds': 43.681461600922816, 'Cash': 4.484867233922344}
2019-03-05 00:00:00
{'Equities': 51.78741498461127, 'Bonds': 43.72648010017677, 'Cash': 4.486104915211967}
2019-03-06 00:00:00
{'Equities': 51.598827522364644, 'Bonds': 43.90264600072124, 'Cash': 4.4985264769141295}
2019-03-07 00:00:00
{'Equities': 51.33182273463001, 'B

2019-07-07 00:00:00
{'Equities': 52.557642356912524, 'Bonds': 43.193388253696455, 'Cash': 4.248969389391021}
2019-07-08 00:00:00
{'Equities': 52.557642356912524, 'Bonds': 43.193388253696455, 'Cash': 4.248969389391021}
2019-07-09 00:00:00
{'Equities': 52.61890317562131, 'Bonds': 43.132473955535026, 'Cash': 4.248622868843662}
2019-07-10 00:00:00
{'Equities': 52.71542093800712, 'Bonds': 43.04787656709657, 'Cash': 4.236702494896318}
2019-07-11 00:00:00
{'Equities': 52.83951235129911, 'Bonds': 42.92231278152178, 'Cash': 4.238174867179128}
2019-07-12 00:00:00
{'Equities': 52.942395492947426, 'Bonds': 42.8319398354007, 'Cash': 4.2256646716518835}
2019-07-13 00:00:00
{'Equities': 52.91576409623365, 'Bonds': 42.86176734023049, 'Cash': 4.222468563535868}
2019-07-14 00:00:00
{'Equities': 52.91576409623365, 'Bonds': 42.86176734023049, 'Cash': 4.222468563535868}
2019-07-15 00:00:00
{'Equities': 52.91576409623365, 'Bonds': 42.86176734023049, 'Cash': 4.222468563535868}
2019-07-16 00:00:00
{'Equities'

{'Equities': 53.606346299845406, 'Bonds': 42.31592875086545, 'Cash': 4.077724949289151}
2019-11-29 00:00:00
{'Equities': 53.606346299845406, 'Bonds': 42.31592875086545, 'Cash': 4.077724949289151}
2019-11-30 00:00:00
{'Equities': 53.51091081670878, 'Bonds': 42.385212730778356, 'Cash': 4.103876452512884}
2019-12-01 00:00:00
{'Equities': 53.51091081670878, 'Bonds': 42.385212730778356, 'Cash': 4.103876452512884}
2019-12-02 00:00:00
{'Equities': 53.49703078664527, 'Bonds': 42.40015725269349, 'Cash': 4.102811960661252}
2019-12-03 00:00:00
{'Equities': 53.20026005160863, 'Bonds': 42.69084105959595, 'Cash': 4.108898888795429}
2019-12-04 00:00:00
{'Equities': 53.4049100598257, 'Bonds': 42.4966230280393, 'Cash': 4.098466912135006}
2019-12-05 00:00:00
{'Equities': 53.47523580507241, 'Bonds': 42.42848833804246, 'Cash': 4.096275856885137}
2019-12-06 00:00:00
{'Equities': 53.725198582245824, 'Bonds': 42.195666773181365, 'Cash': 4.079134644572802}
2019-12-07 00:00:00
{'Equities': 53.644022277041884, 

OutOfBoundsDatetime: Out of bounds nanosecond timestamp 9223372800000000000

In [22]:
sell_amt

23037.328710979928

In [ ]:
portfolio.shares

In [21]:
portfolio.log

{0: ['Buy',
  250.0,
  188.4,
  1.326963906581741,
  '2015-01-01'],
 1: ['Buy',
  250.0,
  82.65,
  3.024803387779794,
  '2015-01-01'],
 2: ['Buy',
  0.17,
  84.08,
  0.002021883920076118,
  Timestamp('2015-02-02 00:00:00', freq='D')],
 3: ['Buy',
  0.162,
  82.65,
  0.0019600725952813067,
  Timestamp('2015-03-02 00:00:00', freq='D')],
 4: ['Buy',
  0.984,
  192.52,
  0.005111157282360274,
  Timestamp('2015-03-23 00:00:00', freq='D')],
 5: ['Buy',
  0.191,
  83.51,
  0.0022871512393725303,
  Timestamp('2015-04-01 00:00:00', freq='D')],
 6: ['Buy',
  0.159,
  82.5,
  0.0019272727272727273,
  Timestamp('2015-05-01 00:00:00', freq='D')],
 7: ['Buy',
  0.166,
  81.9,
  0.002026862026862027,
  Timestamp('2015-06-01 00:00:00', freq='D')],
 8: ['Buy',
  0.902,
  194.27,
  0.0046430225974159675,
  Timestamp('2015-06-22 00:00:00', freq='D')],
 9: ['Buy',
  0.163,
  80.9,
  0.002014833127317676,
  Timestamp('2015-07-01 00:00:00', freq='D')],
 10: ['Buy',
  0.169,
  81.83,
  0.0020652572406207994

In [ ]:
portfolio.asset_split['Bonds']-strat.bond_distribution

In [ ]:
portfolio.asset_split['Bonds']

In [ ]:
246.38667/.502971399

In [ ]:
(246.38667+243.47552)*.002971399

In [ ]:
489.86218/2

In [ ]:
246.38667*.002971399

In [ ]:
output

In [ ]:
portfolio.log

In [ ]:
time_period

In [ ]:
start_date

In [ ]:
# Get the value a few days later

portfolio.get_value('2018-05-01')

In [ ]:
portfolio.get_asset_values('2018-05-01')

In [ ]:
portfolio.asset_values

In [ ]:
for share in portfolio.shares:
    print(share.div.index)

In [ ]:
portfolio.reinvest_divs('2018-05-01')

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas_datareader

In [ ]:
## This is useful:
#https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

In [ ]:
bnd = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp = web.DataReader('VOO', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp.head(3)

In [ ]:
bnd.head(3)

In [ ]:
bnd.loc['2017-07-07']['High']

In [ ]:
# Import dividend records
bnd_div = web.DataReader('BND', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')
sp_div = web.DataReader('VOO', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')

In [ ]:
bnd_div.head(3)

In [ ]:
sp_div.head(3)

In [ ]:
sp_div.value.loc['2020-09-29']

In [ ]:
sp_div[sp_div.index < pd.to_datetime('2016-01-01')]

In [ ]:
sp_div.index

In [ ]:
str('sp')+str('_div')

In [ ]:
Assets = {}
Assets['Bonds'] = bnd
Assets['Equities'] = sp

In [ ]:
Dividends

In [ ]:
# Maybe it is worth merging the dividend and share price files together for easier reference?
# Or build a dictionary to allow easy mapping?

## 1. Calculate Value Given a Start and End Date

In [ ]:
start_date = '2015-11-20'
end_date = 

In [ ]:
def calc_value(share, divs, start, end, initial_value):
    # Take the average of the day's high and low for starting share price
    num_shares = initial_value/np.mean([share.loc[start]['Low'],share.loc[start]['High']])
    divs = divs[(divs.index <= pd.to_datetime(end)) & (divs.index >= pd.to_datetime(start))]
    for date in divs.index:
        price = np.mean([share.High.loc[date], share.Low.loc[date]])
        div_value = divs.value.loc[date]
        num_shares += div_value/price
    output = {}
    output['Value'] = num_shares*np.mean([share.loc[end]['Low'],share.loc[end]['High']])
    output['ROI'] = output['Value']-initial_value
    return(output)
    
    # Will need to factor in other actions later, but this will work for now
    
    


In [ ]:
calc_value(sp,sp_div, '2015-11-20', '2017-11-20',100)

In [ ]:
calc_value(bnd,bnd_div, '2015-11-20', '2017-11-20',100)

In [ ]:
sp.Close.plot()

In [ ]:
bnd.Close.plot()

In [ ]:
# Identify the ROI with different asset distributions

In [ ]:
class Share:

    def __init__(self, name, asset_type):
        """Initialize attributes."""
        self.name = name
        self.type = asset_type
        #self.amount = amount
        
    def get_value(self, date):
        df = web.DataReader(self.name, 'yahoo', start=date, end=date)
        return round(df.loc[date,'Close'],2)
    

In [ ]:
bnd = Share('BND','Bond')

In [ ]:
sp = Share('VOO','Equity')

In [ ]:
class Strategy:
    
    def __init__(self,equity_distribution, bond_distribution, cash_distribution, threshold):
        self.equity_distribution = equity_distribution
        self.bond_distribution = bond_distribution
        self.cash_distribution = 100-(equity_distribution+bond_distribution)
        self.threshold = threshold
    

In [ ]:
class Portfolio:
    
    def __init__(self):
        self.shares = {}
        self.log = {}
        self.cash_bal = 0
        self.asset_split = {'Equities': None,'Bonds': None,'Cash': None}
        self.asset_values = {'Equities': None,'Bonds': None,'Cash': None}
        
    def buy(self, share, amount, date):
        purchase_price = share.get_value(date)
        shares = amount/purchase_price
        # Record the transaction
        self.log[len(self.log)] = ['Buy',share,amount,purchase_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] += shares
        elif share not in self.shares:
            self.shares[share] = shares
            
    def sell(self, share, amount, date):
        sell_price = share.get_value(date)
        shares = amount/sell_price
        # Record the transaction
        self.log[len(self.log)] = ['Sell',share,amount,sell_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] -= shares
        elif share not in self.shares:
            self.shares[share] = shares
        cash_bal += amount
        
            
    def get_value(self,date):
        value = self.cash_bal
        # Iterate through the 'shares' attribute and get the current value for each 'Share' object
        for share in self.shares:
            value += share.get_value(date)*self.shares[share]
        return(value)
    
    def get_asset_values(self,date):
        bond_val = 0
        eq_val = 0
        for share in self.shares:
            if share.type == 'Bond':
                bond_val += share.get_value(date)*self.shares[share]
            elif share.type == 'Equity':
                eq_val += share.get_value(date)*self.shares[share]
        total = bond_val+eq_val+self.cash_bal
        self.asset_values = {'Equities': eq_val,'Bonds': bond_val,'Cash': self.cash_bal}
        self.asset_split = {'Equities': (eq_val/total)*100,'Bonds': (bond_val/total)*100,'Cash': (self.cash_bal/total)*100}
            
        
    

In [ ]:
portfolio = Portfolio()

In [ ]:
portfolio.get_value)

In [ ]:
# Calculate portfolio value

start_date = '2015-11-20'
end_date = '2020-11-20'

portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

portfolio.get_value(end_date)

In [ ]:
bond = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
dates = list(bond.index)

In [ ]:
dates = [x.strftime('%Y-%m-%d') for x in dates]

In [ ]:
# Deploy balancing strategy

strategy = Strategy(75,25,0,1)

# Initial Purchase (manually allocated proportions)
portfolio = Portfolio() 
portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

for date in dates:
    portfolio.get_asset_values(date)
    if portfolio.asset_split['Equities'] > strategy.equity_distribution+strategy.threshold:
        amount_to_sell = (portfolio.asset_split['Equities']-strategy.equity_distribution)*portfolio.asset_values['Equities']
        portfolio.sell(sp,amount_to_sell,date)
        portfolio.buy(bnd,amount_to_sell,date)
        

    

In [ ]:
portfolio.get_asset_values('2019-10-29')

In [ ]:
portfolio.asset_split['Equities']